In [4]:

import pandas as pd
import numpy as np
import xarray as xr
from ecephys import wne, utils, sglxr
import wisc_ecephys_tools as wet

/home/tbugnon/miniconda3/envs/shared_sortings/lib/python3.10/site-packages/seaborn/rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
/home/tbugnon/miniconda3/envs/shared_sortings/lib/python3.10/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


In [14]:
subjectsDir = wet.get_subjects_directory()
projectsFile = wet.get_projects_file()

subjLib = wne.sglx.SubjectLibrary(subjectsDir)
projLib = wne.ProjectLibrary(projectsFile)


In [52]:
anatomyProjectName = "shared_s3"
lfpProjectName = "seahorse"

experiment = "novel_objects_deprivation"

In [60]:
wneAnatomyProject = projLib.get_project("shared_s3")

In [185]:
from pandas.testing import assert_frame_equal

SINGLE_CHANNEL_GAP = 20 # When histo contains only "span", we need to add this between structures to match the channel based def

def save_structure_files(dat, save=False, overwrite=False):
    
    subjectName = dat["subjectName"]
    probe = dat["probe"]
    histo = dat["histo"]
    # hippo = dat.get("hippo", None)
    
    wneSubject = subjLib.get_subject(subjectName)
    wneProject = projLib.get_project(lfpProjectName)
    
    print(histo)

    if "loChIx" in histo:

        # ap_table = wneSubject.get_ap_bin_table(experiment, "full", probe=probe)
        # ap_file = ap_table.iloc[0].path
        # print(ap_file)
        # im = sglxr.ImecMap.from_bin(ap_file)
        lfp_table = wneSubject.get_lfp_bin_table(experiment, "full", probe=probe)
        lfp_file = lfp_table.iloc[0].path
        [nc_file] = wneProject.get_sglx_counterparts(wneSubject.name, [lfp_file], wne.constants.NETCDF_EXT)
        if nc_file.is_file():
            lf = xr.open_dataarray(nc_file)
            y = lf.y.values
        elif lfp_file.is_file():
            im = sglxr.ImecMap.from_bin(lfp_file)
            y = im.y
        else:
            raise FileNotFoundError

        histo['lo'] = y[histo['loChIx']]
        histo['hi'] = y[histo['hiChIx']]
        histo['span'] = histo['hi'] - histo['lo']
        histo = histo[['structure', 'acronym', 'lo', 'hi', 'span']]

    elif "span" in histo and not "lo" in histo:
        
        raise ValueError("Don't use span because errors accumulate... use loChIx/HiChIx")
        
        # span = np.array(histo["span"])
        # lo = np.cumsum(np.insert(span[::-1], 0, 0)[:-1])[::-1] # Cumsum in reverse order, starting from 0
        # print(np.arange(0, SINGLE_CHANNEL_GAP * len(lo) + 1, SINGLE_CHANNEL_GAP))
        # lo = lo + np.arange(0, SINGLE_CHANNEL_GAP * len(lo), SINGLE_CHANNEL_GAP)[::-1] # Add 1-channel gap between successive regions
        # print(np.arange(0, SINGLE_CHANNEL_GAP * len(lo) + 1, SINGLE_CHANNEL_GAP))
        # print(lo)
        # hi = lo + span
        # histo["lo"] = lo
        # histo["hi"] = hi
    
    assert set(histo.columns.values) == {"structure", "acronym", "lo", "hi", "span"}
    assert np.all((histo.hi - histo.lo) == histo.span)

    f = wneAnatomyProject.get_experiment_subject_file(experiment, wneSubject.name, f"{probe}.structures.htsv")
    
    print(histo)
    
    print(f)
    print("file exists: ", f.exists())
    
    if save:
        if f.exists() and not overwrite:
            previous = utils.read_htsv(f)
            assert_frame_equal(histo, previous)
            print("file equal: True")
        
        print("saving")
        utils.write_htsv(histo, f)

Doppio frontal:
- 53 channels outside of the brain
- M2 (Wax and Pax): 2000 um; channels 232 to 331
- Prelimbic(Wax) or A32D and V (Pax): channels 107 to 230
- Medial orbital cortex (MO: Pax Wax): 62 to 105
- Olfactory area or Basal Forebrain (Wax) or Dorsa tenia tecta (Pax: DTT): 1400 um; channels 1 to 60 

In [195]:
# TODO

dat = {
    "subjectName": "CNPIX4-Doppio",
    "probe": "imec1",
    "histo": pd.DataFrame(
        {
            "structure": [
                "Out of brain",
                "Secondary motor cortex",
                "Prelimbic cortex / A32D + A32V",
                "Medial orbital cortex",
                "Olfactory area / Basal forebrain / Dorsia tenia tecta",
            ],
            "acronym": ["OOB", "M2", "PreL", "MO", "DTT"],
            "span": [1040, 2000, 2480, 880, 1180],
        }
    ),
    "hippo": None,
}

save_structure_files(dat, save=False)

                                           structure acronym  span
0                                       Out of brain     OOB  1040
1                             Secondary motor cortex      M2  2000
2                     Prelimbic cortex / A32D + A32V    PreL  2480
3                              Medial orbital cortex      MO   880
4  Olfactory area / Basal forebrain / Dorsia teni...     DTT  1180


ValueError: Don't use span because errors accumulate... use loChIx/HiChIx

Doppio parietal

- PtA: 1440 um; channels 300 to 374
- WM: 220 um; channels 288 to 298
- Hipp: 1880 um; channels 192 to 286 (channel 278 = CA1)
- Thalamus (LPMR): 900 um ; channels 144 to 190
- Thalamus (Po): 1420 um; channels 71 to 142
- Thalamus (VPM): 1000 um; channels 19 to 69
- Hipo (ZI + Lem): 360 um; channels 1 to 18
- Tip: 180 um.

In [187]:
dat = {
    "subjectName": "CNPIX4-Doppio",
    "probe": "imec0",
    "histo": pd.DataFrame({
            "structure": [
                "Medial parietal association cortex",
                "White matter",
                "Hippocampus",
                "Mediorostral lateroposterior thalamic nucleus",
                "Posterior thalamic nucleus",
                "Ventral posteromedial nucleus",
                "Zona incerta",
            ],
            "acronym": ["MPtA", "WM", "HC", "LPMR", "Po", "VPM", "ZI"],
            # "span": [1440, 220, 1880, 900, 1420, 1000, 360],
            "loChIx": [299, 287, 191, 143, 70, 18, 0],
            "hiChIx": [373, 297, 285, 189, 141, 68, 17],
    }),
    "hippo": pd.DataFrame({
            "structure": [
                "CA1 stratum pyramidale and stratum oriens",
                "CA1 stratum radiatum",
                "CA1 stratum lacunosum moleculare",
                "Dorsal dentate gyrus",
                "CA3",
                "Ventral dentate gyrus",
            ],
            'acronym': ['poCA1', 'srCA1', 'slmCA1', 'dDG', 'CA3', 'vDG'],
            'hi': [5760, 5500, 5220, 4940, 4660, 4340],
            'lo': [5500, 5220, 4940, 4660, 4340, 3900],
    }),
}

save_structure_files(dat, save=True, overwrite=True)

                                       structure acronym  loChIx  hiChIx
0             Medial parietal association cortex    MPtA     299     373
1                                   White matter      WM     287     297
2                                    Hippocampus      HC     191     285
3  Mediorostral lateroposterior thalamic nucleus    LPMR     143     189
4                     Posterior thalamic nucleus      Po      70     141
5                  Ventral posteromedial nucleus     VPM      18      68
6                                   Zona incerta      ZI       0      17
                                       structure acronym      lo      hi  \
0             Medial parietal association cortex    MPtA  5980.0  7460.0   
1                                   White matter      WM  5740.0  5940.0   
2                                    Hippocampus      HC  3820.0  5700.0   
3  Mediorostral lateroposterior thalamic nucleus    LPMR  2860.0  3780.0   
4                     Posterior thal

Luigi parietal
- 29 channels outside the brain
- PtA: 1540 um; channels 287 to 355
- WM: 290 um; channels  to 274: 285
- Hipp: 1840 um; channels 180 to 272  (channel 262  = CA1) 
- Thalamus dorsal lateral geniculate nucleus:  960 um ; channels  129 to 177
- Thalamus medial geniculate body: 1230 um ; channels  65 to 127
- Thalamus posterior intralaminar nucleus: 193 um; channels 54 to 63 
- Corticofugal tract and corona radiata:  810 um; channels 12 to 53
- Cornu ammonis: 210 um; channels 1 to 10
- + Tip: 180 um. Cornu ammonis


In [179]:
dat = {
    "subjectName": "CNPIX9-Luigi",
    "probe": "imec0",
    "histo": pd.DataFrame({
            "structure": [
                "Parietal association cortex", # TODO
                "White matter",
                "Hippocampus",
                "Thalamus dorsal lateral geniculate nucleus",
                "Thalamus medial geniculate body",
                "Thalamus posterior intralaminar nucleus",
                "Corticofugal tract and corona radiata",
                "Cornu ammonis",
            ],
            "acronym": [
                "PtA",
                "WM",
                "HC",
                "DLGN",
                "MGB",
                "PIN",
                "CT/CR",
                "CA",
            ], # TODO
            "loChIx": [286, 273, 179, 128, 64, 53, 11,  0],
            "hiChIx": [354, 284, 271, 176, 126, 62, 52, 9],
    }),
    "hippo": None,
}

save_structure_files(dat, save=False)

                                    structure acronym  loChIx  hiChIx
0                 Parietal association cortex     PtA     286     354
1                                White matter      WM     273     284
2                                 Hippocampus      HC     179     271
3  Thalamus dorsal lateral geniculate nucleus    DLGN     128     176
4             Thalamus medial geniculate body     MGB      64     126
5     Thalamus posterior intralaminar nucleus     PIN      53      62
6       Corticofugal tract and corona radiata   CT/CR      11      52
7                               Cornu ammonis      CA       0       9
                                    structure acronym      lo      hi    span
0                 Parietal association cortex     PtA  5720.0  7080.0  1360.0
1                                White matter      WM  5460.0  5680.0   220.0
2                                 Hippocampus      HC  3580.0  5420.0  1840.0
3  Thalamus dorsal lateral geniculate nucleus    DLGN  256

Adrian imec1:

- Channels 354 to 384 are bad channels
- PtA-V2: 1580 um; channels 320 to 353
- WM: 185 um; channels  311 to 319: 
- Hipp: 1960 um; channels 212 to 310  (channel 300  = CA1)
- Thalamus (LPLR):  1115 um ; channels  154 to 210
- External medullary lamina: 160um; channels 144 to 152
- Posteriror thalamic nucleus (Po): 1115 um; channels 86 to 142 
- Medial lemniscus: 246um; channels 72 to 84
- Zona Incerta caudal part: 1082 um; channels 16 to 70
- Sustancia Negra:  245um; channels 4 to 15
- Corticofugal tract corona radiata: 80um;  1:3
- Tip: 180 um. Corticofugal tract


In [189]:
dat = {
    "subjectName": "CNPIX11-Adrian",
    "probe": "imec1",
    "histo": pd.DataFrame({
            "structure": [
                "Parietal association cortex - V2",
                "White matter",
                "Hippocampus",
                "Thalamus (LPLR)",  # TODO
                "External medullary lamina",
                "Posteroior thalamic nucleus",
                "Medial lemniscus",
                "Zona Incerta (caudal part)",
                "Substancia negra",
                "Corticofugal tract corona radiata",
            ],
            "acronym": [
                "PtA-V2",
                "WM",
                "HC",
                "LPLR",
                "EML",
                "Po",
                "ML",
                "ZI",
                "SN",
                "CT"
            ], # TODO
            "loChIx": [319, 310, 211, 153, 143, 85, 71, 15, 3, 0],
            "hiChIx": [352, 318, 309, 209, 141, 151, 83, 69, 14, 2],
    }),
    "hippo": None,
}

save_structure_files(dat, save=True)

                           structure acronym  loChIx  hiChIx
0   Parietal association cortex - V2  PtA-V2     319     352
1                       White matter      WM     310     318
2                        Hippocampus      HC     211     309
3                    Thalamus (LPLR)    LPLR     153     209
4          External medullary lamina     EML     143     141
5        Posteroior thalamic nucleus      Po      85     151
6                   Medial lemniscus      ML      71      83
7         Zona Incerta (caudal part)      ZI      15      69
8                   Substancia negra      SN       3      14
9  Corticofugal tract corona radiata      CT       0       2
                           structure acronym      lo      hi    span
0   Parietal association cortex - V2  PtA-V2  6380.0  7040.0   660.0
1                       White matter      WM  6200.0  6360.0   160.0
2                        Hippocampus      HC  4220.0  6180.0  1960.0
3                    Thalamus (LPLR)    LPLR  3060.0 

Santiago parietal
- Cx: Wax and Pax V1 1434 71 306:377
- White matter  207 10 295:305
- Hipp: Wax and Pax CA1 698 35 260:294
- Hipp: Wax and Pax DG-CA3  1301 65 194:259
- Thalamus: Pax and Wax Dorsal lateral Geniculate Nucleus (DLG) 868 43 149:193
- Thalamus: Pax and Wax Medial Geniculate Nucleus or Body (MGN) 1283 64 84:148
- Wax Suprageniculate Nuc or Zona Incerta caudal; Pax  Zona Incerta caudal or peri peduncular nucleus 490 24 59:83
- Wax Corticofugal tract and/or Substantia Nigra lateral; Pax cerebral Peduncle and/or Substantia Nigra lateral 679 34 24:58
- Hipp: CA3 Cornu Ammonis 430 23 1:23
- Tip 170 0 0


In [100]:
dat = {
    "subjectName": "CNPIX12-Santiago",
    "probe": "imec1",
    "histo": pd.DataFrame(
        {
            "structure": [
                "Out of brain",
                "Medial parietal association cortex",
                "White matter",
                "Hippocampus",
                "Laterodorsal thalamic nucelus / Mediorostral lateroposterior thalamic nucleus",
                "Posterior thalamic nucleus",
                "Ventrolateral thalamic nucleus",
                "Ventromedial thalamic nucleus",
                "Zona incerta",
            ],
            "acronym": ["OOB", "MPtA", "WM", "HC", "LD/LPMR", "Po", "VL", "VM", "ZI"],
            "loChIx": [368, 291, 277, 185, 131, 61, 43, 13, 0],
            "hiChIx": [383, 367, 290, 276, 184, 130, 60, 42, 12],
        }
    ),
    "hippo": pd.DataFrame(
        {
            "structure": [
                "ca1 stratum pyramidale and stratum oriens",
                "ca1 stratum radiatum",
                "ca1 stratum lacunosum moleculare",
                "dorsal dentate gyrus",
                "ca3",
                "ventral dentate gyrus",
            ],
            'acronym': ['poca1', 'srca1', 'slmca1', 'spdg', 'ca3', 'ipdg'],
            'hi': [5540, 5240, 4940, 4680, 4320, 4000],
            'lo': [5240, 4940, 4700, 4500, 4180, 3540],
        }
    )
}

save_structure_files(dat)

                                           structure  acronym      lo      hi  \
0                                       Out of brain      OOB  7360.0  7660.0   
1                 Medial parietal association cortex     MPtA  5820.0  7340.0   
2                                       White matter       WM  5540.0  5800.0   
3                                        Hippocampus       HC  3700.0  5520.0   
4  Laterodorsal thalamic nucelus / Mediorostral l...  LD/LPMR  2620.0  3680.0   
5                         Posterior thalamic nucleus       Po  1220.0  2600.0   
6                     Ventrolateral thalamic nucleus       VL   860.0  1200.0   
7                      Ventromedial thalamic nucleus       VM   260.0   840.0   
8                                       Zona incerta       ZI     0.0   240.0   

     span  
0   300.0  
1  1520.0  
2   260.0  
3  1820.0  
4  1060.0  
5  1380.0  
6   340.0  
7   580.0  
8   240.0  
/Volumes/npx_nfs/shared_s3/novel_objects_deprivation/CNPIX12-Santiago

Francis parietal:
- Secondary visual cortex (Pax and Wax): 1400um; channels 311 to 384
- WM (Pax and Wax): 155um, channels 305 to 310
- CA1 (pax and Wax): 720um; channels 274 to 304
- Dentate Gyrus (Pax and Wax): 1180um; channels 213 to 272
- Lateral posterior thalamic nucleus (Pax and Wax): 982um; channels 170 to 211
- Posterior thalamic nucleus (Pax and Wax): 1500um; channels 93 to 168
- Medial lemniscus (Pax and Wax): 240 um; channels 80 to 91
- Zona Incerta (Pax and Wax): 900um; channels 35 to 78
- Substantia nigra (Pax and Wax): 140um; channels 27 to 33
- Corticofugal tract (Wax) or cerebral peduncle (Pax): 500um; channels 1 to 25

In [193]:
dat = {
    "subjectName": "CNPIX14-Francis",
    "probe": "imec1",
    "histo": pd.DataFrame(
        {
            "structure": [
                "Secondary visual cortex",
                "White matter",
                "CA1",
                "Dentate gyrus",
                "Lateral posterior thalamic nucleus",
                "Posterior thalamic nucleus",
                "Medial lemniscus",
                "Zona incerta",
                "Substantia nigra",
                "Corticofugal tract",
            ],
            "acronym": ["V2", "WM", "CA1", "DG", "LP", "PO", "ML", "ZI", "SN", "CFT"],
            # "span": [1400, 155, 720, 1180, 982, 1500, 240, 900, 140, 500],
            "loChIx": [310, 304, 273, 212, 169, 92, 79, 34, 26, 0],
            "hiChIx": [383, 309, 303, 271, 210, 167, 90, 77, 32, 24],
        }
    ),
    "hippo": pd.DataFrame(
        {
            "structure": [
                "CA1 stratum pyramidale and stratum oriens",
                "CA1 stratum radiatum",
                "CA1 stratum lacunosum moleculare",
                "Dorsal dentate gyrus",
                "CA3/DG",
                "Ventral dentate gyrus",
            ],
            'acronym': ['poCA1', 'srCA1', 'slmCA1', 'dDG', 'CA3-DG', 'vDG'],
            'hi': [6040, 5760, 5500, 5220, 4940, 4700],
            'lo': [5760, 5500, 5220, 4940, 4700, 4140],
        }
    ),
}

save_structure_files(dat, save=False, overwrite=False)

                            structure acronym  loChIx  hiChIx
0             Secondary visual cortex      V2     310     383
1                        White matter      WM     304     309
2                                 CA1     CA1     273     303
3                       Dentate gyrus      DG     212     271
4  Lateral posterior thalamic nucleus      LP     169     210
5          Posterior thalamic nucleus      PO      92     167
6                    Medial lemniscus      ML      79      90
7                        Zona incerta      ZI      34      77
8                    Substantia nigra      SN      26      32
9                  Corticofugal tract     CFT       0      24
                            structure acronym      lo      hi    span
0             Secondary visual cortex      V2  6200.0  7660.0  1460.0
1                        White matter      WM  6080.0  6180.0   100.0
2                                 CA1     CA1  5460.0  6060.0   600.0
3                       Dentate gyrus 

Claude parietal
- Cx: Wax and Pax V1, 1434,  306:377
- White matter , 207, 295:305,
- Hipp: Wax and Pax CA1, 698, 260:294,
- Hipp: Wax and Pax DG-CA3 , 1301, 194:259,
- Thalamus: Pax and Wax Dorsal lateral Geniculate Nucleus (DLG), 868, 149:193,
- Thalamus: Pax and Wax Medial Geniculate Nucleus or Body (MGN), 1283, 84:148,
- Wax Suprageniculate Nuc or Zona Incerta caudal; Pax  Zona Incerta caudal or peri peduncular nucleus, 490, 59:83,
- Wax Corticofugal tract and/or Substantia Nigra lateral; Pax cerebral Peduncle and/or Substantia Nigra lateral, 679, 24:58,
- Hipp: CA3 Cornu Ammonis, 430, 1:23,
- Tip, 170, 0,


In [198]:
dat = {
    "subjectName": "CNPIX15-Claude",
    "probe": "imec1",
    "histo": pd.DataFrame(
        {
            "structure": [
                "Cx: Wax and Pax V1",
                "White matter",
                "Hipp: Wax and Pax CA1",
                "Hipp: Wax and Pax DG-CA3",
                "Thalamus: Pax and Wax Dorsal lateral Geniculate Nucleus (DLG)",
                "Thalamus: Pax and Wax Medial Geniculate Nucleus or Body (MGN)",
                "Wax Suprageniculate Nuc or Zona Incerta caudal; Pax  Zona Incerta caudal or peri peduncular nucleus",
                "Wax Corticofugal tract and/or Substantia Nigra lateral; Pax cerebral Peduncle and/or Substantia Nigra lateral",
                "Hipp: CA3 Cornu Ammonis",
            ],
            "acronym": ["Cx - V1", "WM", "CA1", "DG-CA3", "DLG", "MGN", "SN/ZI/PPN", "CFT/SNL/CP/SNL", "CA3"],
            "loChIx": [305, 294, 259, 193, 148, 83, 58, 23, 0],
            "hiChIx": [376, 304, 293, 258, 192, 147, 82, 57, 22],
        }
    ),
    "hippo": None,
}

save_structure_files(dat, save=True, overwrite=False)

                                           structure         acronym  loChIx  \
0                                 Cx: Wax and Pax V1         Cx - V1     305   
1                                       White matter              WM     294   
2                              Hipp: Wax and Pax CA1             CA1     259   
3                           Hipp: Wax and Pax DG-CA3          DG-CA3     193   
4  Thalamus: Pax and Wax Dorsal lateral Geniculat...             DLG     148   
5  Thalamus: Pax and Wax Medial Geniculate Nucleu...             MGN      83   
6  Wax Suprageniculate Nuc or Zona Incerta caudal...       SN/ZI/PPN      58   
7  Wax Corticofugal tract and/or Substantia Nigra...  CFT/SNL/CP/SNL      23   
8                            Hipp: CA3 Cornu Ammonis             CA3       0   

   hiChIx  
0     376  
1     304  
2     293  
3     258  
4     192  
5     147  
6      82  
7      57  
8      22  
                                           structure         acronym      lo  \